In [1]:
! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain

In [3]:
!pip install python-dotenv

In [5]:
from dotenv import load_dotenv
import os 
load_dotenv()
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_SECRET_KEY")
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_SECRET_KEY")

In [7]:
#the documents 
question = "What kind of pets do I like?"
document = "My favorite pet is a dog."

In [9]:
#returning number of tokens in a text string 
import tiktoken 
def num_tokens_from_string(string: str, encoding_name: str):
    #getting the encoder/tokenizer object
    encoding = tiktoken.get_encoding(encoding_name)
    #encoding the string into to tokens, and then figuring out how many tokens there are 
    num_tokens = len(encoding.encode(string))
    return num_tokens 
#getting the number of tokens in the question 
num_tokens_from_string(question, "cl100k_base")
#can understand how large the documents we want to feed in are 

8

In [11]:
#getting a fixed length vector representation for both the document and the query/question 
from langchain_openai import OpenAIEmbeddings
embd = OpenAIEmbeddings()
query_result = embd.embed_query(question)
document_result = embd.embed_query(document)
#getting the length of the vector representation for the query
len(query_result)

1536

In [13]:
#cosine similairty is used to compare how similair to vectors are 
#it is done by calculating cosine of the angle between them 

In [15]:
import numpy as np
def cosine_similairty(vec1, vec2): 
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2) 
similairity = cosine_similairty(query_result, document_result)
print(similairity)

0.881331653349696


In [17]:
#loading in a document 
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [19]:
#now we are going to be splitting the document 
from langchain.text_splitter import RecursiveCharacterTextSplitter 
#configuring the splits 
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    #maximum size for each token 
    chunk_size = 300, 
    #each chunk will overlap with the previous one by 50 tokens
    chunk_overlap = 50)


#actually splitting 
splits = text_splitter.split_documents(blog_docs)

In [21]:
#retrieval stage 

In [23]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma 
#taking each split, embedding it with OpenAI embeddings into a vector representation
#storing it in the vectorstore 
vectorstores = Chroma.from_documents(documents=splits, embedding = OpenAIEmbeddings())
#turning the vectorstore into a an retriever object 
#will retrieve the most relevant document chunks based on the similairty score
#k:1 in the search_kwards means that retrieve only the top 1 most relevant document based on the cosine similairty 
retriever = vectorstores.as_retriever(search_kwargs={"k": 1})

In [25]:
#example of getting relevant documents 
#feeding in a certain question too
docs = retriever.get_relevant_documents("What is task deocomposition?")

C:\Users\jesun\AppData\Local\Temp\ipykernel_17184\1550653262.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents("What is task deocomposition?")


In [27]:
#generation stage 

In [31]:
from langchain_openai import ChatOpenAI 
from langchain.prompts import ChatPromptTemplate 
#this is the template which will be used to guide the llm 
#the question should be answered based on the given context only nothing outside of that 
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

#now we can create an actual LangChain ChatPromptTemplate obejct based off the string 
#this will make it structured and will allow it to accept values for the context and the actual question 
prompt = ChatPromptTemplate.from_template(template)
#can now show this newly made object
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'), additional_kwargs={})])

In [33]:
#now we can define the llm model 
#temperature=0 means that the answer generated will be the most confident answer 
llm = ChatOpenAI(model_name ="gpt-4o-mini", temperature=0)

In [35]:
#we are connecting the prompt we previously defined to the llm we just made, forming a chain 
chain = prompt | llm 

In [39]:
#now we can invoke the chain 
#using the information in the docs that we retrieved before to answer the question 
chain.invoke({"context":docs,"question":"What is task decomposition?"})

AIMessage(content='Task decomposition is the process of breaking down a complicated task into smaller, more manageable steps. It involves identifying the individual components or subgoals necessary to achieve the overall task. Techniques such as Chain of Thought (CoT) and Tree of Thoughts are used to facilitate this process, allowing models to think step by step and explore multiple reasoning possibilities. Task decomposition can be achieved through simple prompting, task-specific instructions, or human inputs.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 86, 'prompt_tokens': 316, 'total_tokens': 402, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BeSPZKxweQHrFeCpcnyH0zXOQ4CCl', 'service